# Исследование надежности заемщиков

## Описание проекта
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.
## Описание данных
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита


In [1]:
import pandas as pd

try:
    data = pd.read_csv('data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [3]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [4]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

Исправим отрицательное количество дней трудового стажа в столбце `days_employed`

In [5]:
data['days_employed'] = data['days_employed'].abs()

In [6]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть.

In [7]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.

In [8]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [9]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [10]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [11]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.

In [12]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв.

In [13]:
data['education'] = data['education'].str.lower()

In [14]:
data.duplicated().sum()

71

In [15]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [16]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [17]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [18]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Сформируем на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [19]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [20]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Исследование данных

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [21]:
data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose', 'total_income_category', 'purpose_category'],
      dtype='object')

In [22]:
print(data[['children', 'debt']].corr(method='spearman', min_periods=3))
data.groupby('children').agg({'debt':['mean','count']})

          children      debt
children  1.000000  0.028706
debt      0.028706  1.000000


debt       
              mean  count
children                 
0         0.075438  14091
1         0.092346   4808
2         0.094542   2052
3         0.081818    330
4         0.097561     41
5         0.000000      9

**Вывод:** Заёмщики без детей реже имеют долги, чем остальные группы. Заёмщики с более чем 3 детьми слишком малочисленны.

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
# Переменная качественная, поэтому не используем коэффициент корреляции
data.groupby('family_status').agg({'debt':['mean','count']})

debt       
                           mean  count
family_status                         
Не женат / не замужем  0.097639   2796
в разводе              0.070648   1189
вдовец / вдова         0.066246    951
гражданский брак       0.093130   4134
женат / замужем        0.075606  12261

**Вывод:** Заёмщики в статусе 'не женат/не замужем' и 'гражданский брак' более склонны к просрочке платежей.

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
print(data[['total_income', 'debt']].corr(min_periods=3))
data.groupby('total_income_category').agg({'debt':['mean','count']})

              total_income      debt
total_income      1.000000 -0.012671
debt             -0.012671  1.000000


debt       
                           mean  count
total_income_category                 
A                      0.080000     25
B                      0.070602   5014
C                      0.084982  15921
D                      0.060172    349
E                      0.090909     22

**Вывод:** Заёмщики в категориях A, D и E слишком малочисленны. Взаимосвязь практически отсутствует.

#### Как разные цели кредита влияют на его возврат в срок?

In [25]:
data.groupby('purpose_category').agg({'debt':['mean','count']})

debt       
                              mean  count
purpose_category                         
операции с автомобилем    0.093480   4279
операции с недвижимостью  0.072551  10751
получение образования     0.092528   3988
проведение свадьбы        0.079118   2313

**Вывод:** Заёмщики, берущие деньги на операции с автомобилем и получение образования, более склонны к просрочке платежей.

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

In [26]:
import pandas as pd

def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')
    
data['purpose_category'] = data['purpose'].apply(categorize_purpose)
data['days_employed'] = data['days_employed'].abs()
data_no_income = data[data['total_income'].isna()==True]
print('')
print('Проверяем гипотезу, что это кредиты на образование (не требуют 2-НДФЛ):')
print(data_no_income['purpose_category'].value_counts())
print('')
print('Проверяем гипотезу, что люди без стажа:')
print(data['days_employed'].median())
print(data_no_income['days_employed'].median())
print('')
print('Проверяем гипотезу, что это безработные:')
print(data_no_income['income_type'].value_counts())


Проверяем гипотезу, что это кредиты на образование (не требуют 2-НДФЛ):
purpose_category
операции с недвижимостью    1082
получение образования        425
операции с автомобилем       418
проведение свадьбы           249
Name: count, dtype: int64

Проверяем гипотезу, что люди без стажа:
2194.220566878695
nan

Проверяем гипотезу, что это безработные:
income_type
сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: count, dtype: int64


*Ответ:* Поскольку оба столбца, где были пропуски (стаж и доход) связаны с данными трудовой деятельности, то возможно, что у заёмщика отсутствовали при себе какие-либо документы, либо данные не удалось добыть из бюро кредитных историй

- Гипотеза, что это скорее всего студенты, отвергнута
- Гипотеза, что это скорее всего безработные, отвергнута
- Поскольку это занятые (среди них нет безработных и студентов), то скорее всего, они работают "в серую" и не имеют как сведений о доходах, так и сведений о стаже

### Общий вывод

В результате анализа данных по заёмщикам удалось выяснить, что на надёжность влияют:
- наличие детей (заёмщики без детей более надёжны)
- семейное положение (более надёжны заёмщики в браке и вдовы/вдовцы)
- цель получения кредита (более надёжны заёмщики, берущие деньги на операции с недвижимостью и свадьбы)
<br> 
Заёмщики в категориях A, D и E слишком малочисленны. Взаимосвязь с наличием долгов по сегментам практически отсутствует. Нам следует пересмотреть присвоение рейтинга.